In [ ]:
import tensorflow as tf
import os
from scipy.special import softmax
from PIL import Image

In [ ]:
test_data = '/content/drive/MyDrive/CV_project/data/안구질환/개_train_test_val_squared/test'

In [ ]:
model_path = '/content/drive/MyDrive/CV_project/안구질환_모델/Resnet_squared_e30_b32_model.tf'
model = tf.keras.models.load_model(model_path)

In [ ]:
# 라벨 한국어 번역
labels = ['결막염',
          '궤양성 각막질환',
          '백내장',
          '비궤양성 각막질환',
          '색소 침착성 각막염',
          '안검내반증',
          '안검염',
          '안검 종양',
          '유루증',
          '정상',
          '핵경화']

In [ ]:
import numpy as np

In [ ]:
def inference(file_path):
    img = Image.open(file_path)
    img = img.resize((224, 224))
    img = np.array(img)

    img = img / 255.
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img, verbose=0)
    return pred

In [ ]:
predictions = {'target':[], 'pred':[], 'prob':[]}
for folder in os.listdir(test_data):
    print(folder)
    for file in os.listdir(os.path.join(test_data, folder)):
        pred = inference(os.path.join(test_data, folder, file))
        predictions['target'].append(folder)
        predictions['pred'].append(labels[pred.argmax()])
        predictions['prob'].append(pred.tolist())
        if len(predictions['target']) % 1000 == 0:
            print(len(predictions['target']), "predictions done")

안검내반증
정상
1000 predictions done
2000 predictions done
3000 predictions done
4000 predictions done
5000 predictions done
6000 predictions done
7000 predictions done
8000 predictions done
9000 predictions done
핵경화
안검종양
백내장
비궤양성각막질환
궤양성각막질환
색소침착성각막염
결막염
10000 predictions done
안검염
유루증


In [ ]:
from collections import Counter

Counter(predictions['target'])

Counter({'안검내반증': 675,
         '정상': 8515,
         '핵경화': 72,
         '안검종양': 11,
         '백내장': 107,
         '비궤양성각막질환': 77,
         '궤양성각막질환': 415,
         '색소침착성각막염': 45,
         '결막염': 111,
         '안검염': 129,
         '유루증': 841})

In [ ]:
Counter(predictions['pred'])

Counter({'유루증': 810,
         '안검내반증': 582,
         '정상': 8727,
         '핵경화': 97,
         '안검염': 83,
         '안검 종양': 7,
         '궤양성 각막질환': 381,
         '결막염': 70,
         '백내장': 112,
         '비궤양성 각막질환': 95,
         '색소 침착성 각막염': 34})

In [ ]:
import unicodedata
target = [unicodedata.normalize('NFC', t) for t in predictions['target']]
predictions['target'] = target

In [ ]:
pred = [unicodedata.normalize('NFC', p) for p in predictions['pred']]
predictions['pred'] = pred

### D2 폰트 적용

In [ ]:
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt

font_files = font_manager.findSystemFonts(fontpaths='/content/drive/MyDrive/Pal-ette/D2Coding')
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

plt.rc('font', family='D2Coding')

### confusion matrix 생성 및 시각화

In [ ]:
# confusion matrix 생성 및 시각화
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

cm = confusion_matrix(predictions['target'], predictions['pred'], normalize='true')
hm = sns.heatmap(cm, annot=True, fmt='.2f', cmap='flare_r')
hm.set_xlabel('예측값', fontsize=10)
hm.set_ylabel('실제값', fontsize=10)
hm.set_xticklabels(labels=labels, fontsize=10, rotation=45, ha='right', rotation_mode='anchor')
hm.set_yticklabels(labels=labels, fontsize=10, rotation=0)

plt.show()

NameError: name 'predictions' is not defined

In [ ]:
from sklearn.metrics import classification_report

cr = classification_report(predictions['target'], predictions['pred'])
print(cr)

              precision    recall  f1-score   support

         결막염       0.66      0.41      0.51       111
     궤양성각막질환       0.94      0.87      0.90       415
         백내장       0.79      0.83      0.81       107
    비궤양성각막질환       0.47      0.58      0.52        77
    색소침착성각막염       0.35      0.27      0.30        45
       안검내반증       0.53      0.46      0.49       675
         안검염       0.45      0.29      0.35       129
        안검종양       0.43      0.27      0.33        11
         유루증       0.61      0.59      0.60       841
          정상       0.96      0.98      0.97      8515
         핵경화       0.55      0.74      0.63        72

    accuracy                           0.89     10998
   macro avg       0.61      0.57      0.58     10998
weighted avg       0.89      0.89      0.89     10998



In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import numpy as np

legend = []

for i in range(len(labels)):
    target = np.array(predictions['target'])==i
    target = target.astype(int)
    probs = np.array(predictions['probs'])[:, i]

    precision, recall, thresholds = precision_recall_curve(target, probs)
    pr_auc = auc(recall, precision)
    plt.plot(recall, precision)
    legend.append(labels[i] + '(AUC = ' + str(round(pr_auc,4)) + ')')

plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(legend, loc='lower left')
plt.show()

KeyError: 'probs'